In [5]:
import pandas
%reset -f

import sys
sys.path.append("../data/exadata/parquet_dataset/query_tool")

from toolz import pipe
import pandas as pd
import numpy as np
import datetime
import torch
from torch.utils.data import Dataset
from sklearn.linear_model import LinearRegression
from query_tool import M100DataClient
from matplotlib import pyplot as plt
from models import LinearNN, StateEstimator

dataset_path = "../data/m100"
client = M100DataClient(dataset_path)



In [1]:
"""Cell purpose: pull desired features from plugins"""

#For now I think it's necessary to do it this way because of memory issues
def extract_features(feature_list, n_rows, columns, year_month):
    df = pd.DataFrame()
    for f in feature_list: 
        d = client.query(f, 
                      columns=columns,
                      year_month=year_month)
        selected_rows = d if df.shape[0] < n_rows else d[:n_rows] #could alos randomnly sample from df
        df = pd.concat([df,selected_rows]).pivot(index="timestamp", columns="node")
        print(df)
        print(pd.concat([df,selected_rows]).pivot(index="timestamp", columns="node", values="value"))
        del d; del selected_rows #save memory. Gc might handle on its own but idk, doesn't hurt
    return df

n_rows = 50
year_month = ["22-02"]
#vertiv plugin collects data for AC cooling 
# vertiv_df = client.query(["Supply_Air_Temperature", "Return_Air_Temperature"],
#                   columns=['timestamp','value'],
#                   year_month=["22-02"])



#ipmi pluging collects physical/hardware properties of nodes
ipmi_features = ["ambient", "gpu0_core_temp", "p0_core0_temp"]
ipmi_columns = ["timestamp", "node", "value"]
ipmi_df = extract_features(ipmi_features, n_rows, ipmi_columns, year_month)
# #ganglia collects metrics on utilization of hardware
# ganglia_df = client.query(["load_fifteen"],
#                   columns=['timestamp', "node", 'value'],
#                   year_month=["22-02"])

# logics collects power consumption data from equipment and devices
# logics_df = client.query(["Tot"],
#                   columns=['timestamp', "device", "panel", 'value'],
#                   year_month=["22-02"])

#schneider collects data on liquid cooling system
# schneider_df = client.query(["Tot"],
#                   columns=['timestamp', "node", 'value'],
#                   year_month=["22-02"])

#weather data
# weather_df = client.query(["temp"],
#                   columns=['timestamp', 'value'],
#                   year_month=["22-02"])


ipmi_df.head()


NameError: name 'pd' is not defined

In [5]:
# dfs = [weather_df, schneider_df, logics_df, ganglia_df, ipmi_df, vertiv_df]
# plugin_dfs = [ipmi_df, vertiv_df, weather_df]
plugin_dfs = [ipmi_df]
for d in plugin_dfs:
    print(d.shape)
    print(d.pivot(index="timestamp").head(3))

(118500161, 3)
                  timestamp node  value
0 2022-02-07 18:56:20+00:00   44   16.6
1 2022-02-07 18:56:40+00:00   44   16.6
2 2022-02-07 18:57:00+00:00   44   16.4
(2297640, 2)
                  timestamp      value
0 2022-02-26 23:00:00+00:00  24.299999
1 2022-02-26 23:00:00+00:00  24.200001
2 2022-02-26 23:00:00+00:00  22.799999
(3400, 3)
                  timestamp node  value
0 2022-02-27 23:10:10+00:00  NaN   5.08
1 2022-02-27 23:30:00+00:00  NaN   4.79
2 2022-02-27 23:40:00+00:00  NaN   4.43


In [6]:
"""Cell purpose: preprocess and/or merge data as desired"""
df = pd.concat(plugin_dfs)
# print(merged_df["timestamp"].astype(np.int64) * 10e-9)
# merge_df["timestamp"] = merge_df["timestamp"].apply(lambda ts: ts.as_unit("ns"))
df.head(10)

,timestamp,node,value
0,2022-02-07 18:56:20+00:00,44,16.6
1,2022-02-07 18:56:40+00:00,44,16.6
2,2022-02-07 18:57:00+00:00,44,16.4
3,2022-02-07 18:57:20+00:00,44,16.4
4,2022-02-07 18:57:40+00:00,44,16.4
5,2022-02-07 18:58:00+00:00,44,16.4
6,2022-02-07 18:58:20+00:00,44,16.4
7,2022-02-07 18:58:40+00:00,44,16.4
8,2022-02-07 18:59:00+00:00,44,16.4
9,2022-02-07 18:59:20+00:00,44,16.4


In [ ]:
# Will have to make work with timestamp indexign later
# class M100Data(Dataset):
#     def __init__(self, df, transform = None, label_transform = None):
#         self.df = df
#         self.transform = transform
# 
#     def __len__(self):
#         return len(self.df.shape[0])
# 
#     def __getitem__(self, idx):
#         return feat, label
    
    

In [ ]:
"""Cell purpose: train"""
model = LinearRegression().fit(merged_df["air_temp"].to_numpy().reshape(-1,1), merged_df["ambient_temp"].to_numpy().reshape(-1,1))

In [ ]:
"""Cell purpose: inference"""
preds = model.predict(test_df["air_temp"].to_numpy().reshape(-1,1))


In [ ]:
"""Cell purpose: benchmarking"""

labels = test_df["ambient_temp"].to_numpy().reshape(-1,1)


timesteps = [t for t in range(preds.shape[0])]
colors = iter(plt.cm.rainbow(np.linspace(0, 0.5, 2)))
plt.scatter(preds, labels, c=["blue", "red"])

plt.legend(loc="upper left")
plt.xlabel("Time")
plt.ylabel("Ambient temperature")
plt.grid()
plt.show()



In [ ]:
""
